# **공연별 예매된 좌석, 예매되지 않은 좌석 테이블 구조 만들기**

In [1]:
import pandas as pd 
import sys
import numpy as np
import time
import warnings

warnings.filterwarnings(action = 'ignore')

In [13]:
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_information_angle_ver2.csv",encoding = 'utf-8')
df = pd.read_parquet("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/dataframe_reduced_size.parquet")

In [14]:
seat_information.head(3)

,층,블록,열,넘버,전체_좌석,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,좌우시야각,상하시야각
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,69.517555,3.436010
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,68.459024,3.520697
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,67.347261,3.608334


In [15]:
# merge 하기 위해 전체_좌석 컬럼을 seat 라는 컬럼 명으로 변경시커주자 

seat_information = seat_information.rename(columns = {'전체_좌석':'seat'})

display(seat_information.head())

,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,좌우시야각,상하시야각
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,69.517555,3.436010
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,68.459024,3.520697
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,67.347261,3.608334
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6,1440.234009,21.009736,35.069575,66.199420,3.702327
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5,1403.862529,23.314833,35.426958,65.044882,3.798387


In [16]:
print('좌석 정보 데이터 프레임 : ', seat_information.shape)
print('메모리 정리 된 데이터 프레임 :', df.shape)

좌석 정보 데이터 프레임 :  (2505, 14)
메모리 정리 된 데이터 프레임 : (236966, 33)


# **해야 할 일**

각 공연 일, performance code 별 예매 된 좌석에 대해서는 예매 여부를 담은 테이블을 만들어야 함

In [17]:
print('유니크한 공연일 개수 : ',df['전체공연시간'].nunique())

유니크한 공연일 개수 :  162


# **빅 콘테스트 의문사항**

In [18]:
unique_combinations = df.groupby(['genre','performance_code', '전체공연시간']).size().reset_index(name='count')

cond = unique_combinations['count'] > 2505

unique_combinations[cond]

,genre,performance_code,전체공연시간,count
8107,클래식,902,2019-03-14 20:00:00,2580
8922,클래식,916,2019-05-19 17:00:00,2961
10526,클래식,989,2023-05-21 17:00:00,2718
11483,클래식,1182,2022-12-20 19:30:00,2558
16756,클래식,1725,2021-03-31 19:30:00,3030
16869,클래식,1828,2019-07-13 17:00:00,2723
17027,클래식,1869,2019-07-02 20:00:00,2574
18919,클래식,2158,2022-10-20 20:00:00,2743


전체 공연 좌석이 2505 개인데 2505개보다 많은 좌석이 판매된 경우가 있음

살펴보니 같은 좌석에서 다른 사람이 다른 시간에 예매를 한 것으로 나옴

이렇게 되면 해당 공연의 예매율은 100% 를 넘어갈텐데

우선 그룹핑 할때는 이런 일이 없도록 drop_duplicate 를 시행해주자

# **좌석 좌표와 병합 후 없는 좌석은 예매 여부 0으로 하기**

# **병합 전 seat_information.csv 파일 수정**

In [19]:
cond = seat_information['블록'].str.contains('BOX')

# Box석의 경우 BOX 의 번호가 열이 되도록 수정

seat_information.loc[cond, '열'] = seat_information.loc[cond, '블록'].map(lambda x: x[3:])
seat_information.loc[cond, '블록'] = seat_information.loc[cond, '블록'].map(lambda x: x[:3])

In [20]:
print('맨 위 5개의 행')
display(seat_information.head())

print('맨 아래 5개의 행')
display(seat_information.tail())

맨 위 5개의 행


,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,좌우시야각,상하시야각
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,69.517555,3.436010
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,68.459024,3.520697
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,67.347261,3.608334
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6,1440.234009,21.009736,35.069575,66.199420,3.702327
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5,1403.862529,23.314833,35.426958,65.044882,3.798387


맨 아래 5개의 행


,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,좌우시야각,상하시야각
2500,2층,BOX,6,2,2층 BOX6 2,-1432,505,383,2층 BOX1 1,1565.994253,21.173142,34.169657,-70.574617,-14.156604
2501,2층,BOX,6,3,2층 BOX6 3,-1500,565,428,2층 BOX1 4,1659.038577,21.144470,32.190452,-69.360281,-14.950280
2502,2층,BOX,6,4,2층 BOX6 4,-1454,617,428,2층 BOX1 3,1636.456232,23.354990,31.913243,-67.006224,-15.161505
2503,2층,BOX,6,5,2층 BOX6 5,-1522,677,483,2층 BOX1 6,1734.388077,23.078793,30.097223,-66.020049,-16.169769
2504,2층,BOX,6,6,2층 BOX6 6,-1568,624,483,2층 BOX1 5,1755.360077,21.128253,30.390770,-68.299463,-15.971384


In [21]:
class seat_merging:
    
    def __init__(self,data,seat):
        '''
        예매 된 공연 테이블과 좌석 정보를 담은 테이블을 가져오기
        '''
        self.df = data 
        self.seat = seat
        
        self.seat['예매여부'] = 1 # merging 후 특정 조건에 따라 예매 여부를 0으로 mapping 하여 수정 할 예정 
        
        
        self.genres_df_dict = {}
        self.genres_merged_dict = {}
        self.genres_num_count = {}

            
    def merging_seat_information(self):
        
        # 병합 후 채워야 하는 컬럼들 
        
        arr = ['play_date','play_st_time','ticket_cancel','performance_code',
                    'pre_open_date','open_date','genre','place','running_time','intermission',
                    '공연연도','공연월','공연일','공연연월','전체공연시간']
                
        unique_performance = sorted(self.df['전체공연시간'].unique())        

        merging_df_list = []
        
        for uni in unique_performance: # 장르 별, 공연 별로 merge 시키기
            
            cond = self.df['전체공연시간'] == uni
            
            cond_df = self.df[cond]
            
            cancle_cond = cond_df['ticket_cancel'] == 1 # 티켓을 취소한 경우도 제거해야 한다 
            
            cond_df = cond_df[~cancle_cond]
            
            # 좌석의 중복값을 제거해야함 
            # 전체 거래시간 별로 오름차순 정렬 후 마지막에 중복값중 마지막 중복값 제외 drop 하기
            # 마지막 거래가 실제 거래라고 생각하기로 함
            
            cond_df = cond_df.sort_values(by = '전체거래시간')
            cond_df = cond_df.drop_duplicates(subset = 'seat', keep = 'last')
            
            # seat information 과 유니크한 공연 별 데이터 프레임을 병합 
            # 이렇게 하게 되면 merging_df 는 2505 행의 데이터프레임이 완성 될 것임 
            merging_df = pd.merge(self.seat[['seat','층', '블록', '열', '넘버', 'X', 'Y', 'Z', '대칭점', '좌우시야각',
                                            '상하시야각', '좌우면적시야각', '상하면적시야각','무대까지의 거리','예매여부']], cond_df, on = 'seat', how = 'left')
            
            
            not_purcahce_cond = merging_df['전체공연시간'].isna()
            
            merging_df.loc[not_purcahce_cond, '예매여부'] = 0 # 예매가 안됐던 경우에 대해서는 예매 여부를 0으로 변경하기 
            
            for fill_col in arr: 
                if merging_df[fill_col].notna().any():
                    original_value = merging_df.loc[merging_df[fill_col].notna(), fill_col].iloc[0]
                    merging_df[fill_col] = merging_df[fill_col].fillna(original_value)
                
            merging_df_list.append(merging_df) # merging_df_list 에 차곡 차곡 2505 석의 공연 별로 채워주자 
                    
        result = pd.concat(merging_df_list)
        
        # 장르하고 place 는 drop 하자 이미 클래식과 콘서트홀만 연동된다
        
        result = result.drop(columns = ['genre', 'place'], axis = 1)
        
        return result

In [22]:
merging_method = seat_merging(df.drop(['층','석','세부좌석'],axis = 1),
                                seat_information) # 기존 df 의 층,석,세부좌석이 seat_information 과 같은 내용이니 하나는 drop 하자

In [23]:
merging_df = merging_method.merging_seat_information()

In [24]:
print('예매 여부가 1인 경우')
display(merging_df[merging_df['예매여부'] == 1].sample(5))

print('예매 여부가 0인 경우')
display(merging_df[merging_df['예매여부'] == 0].sample(5))

예매 여부가 1인 경우


,seat,층,블록,열,넘버,X,Y,Z,대칭점,좌우시야각,...,open_date,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
430,1층 B블록9열 1,1층,B블록,9,1,1068,1411,50,1층 D블록9열 11,37.122443,...,2022-09-24,100.0,15.0,Y,2022.0,10.0,2022-10-30,2022-10-01,2022-10-30 17:00:00,2022-10-03 16:04:00
2276,3층 B블록5열 7,3층,B블록,5,7,1086,2811,1328,3층 F블록5열 1,21.123510,...,2019-11-21,105.0,15.0,Y,2020.0,1.0,2020-01-11,2020-01-01,2020-01-11 17:00:00,2019-12-17 07:39:00
1960,2층 B블록7열 14,2층,B블록,7,14,424,2915,888,2층 D블록7열 1,8.275893,...,2023-01-08,120.0,0.0,N,2023.0,2.0,2023-02-10,2023-02-01,2023-02-10 19:30:00,2023-01-13 13:06:00
772,1층 B블록15열 6,1층,B블록,15,6,926,2036,172,1층 D블록15열 8,24.456675,...,2022-07-31,100.0,15.0,N,2022.0,10.0,2022-10-29,2022-10-01,2022-10-29 17:00:00,2022-09-19 11:47:00
383,1층 B블록8열 11,1층,B블록,8,11,524,1389,30,1층 D블록8열 1,20.668974,...,2019-01-27,120.0,20.0,Y,2019.0,5.0,2019-05-19,2019-05-01,2019-05-19 17:00:00,2019-01-25 10:01:00


예매 여부가 0인 경우


,seat,층,블록,열,넘버,X,Y,Z,대칭점,좌우시야각,...,open_date,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
2010,2층 A블록8열 7,2층,A블록,8,7,1441,2766,923,2층 E블록8열 2,27.518113,...,2022-03-07,120.0,15.0,NaN,2022.0,4.0,2022-04-06,2022-04-01,2022-04-06 19:30:00,NaT
530,1층 E블록10열 9,1층,E블록,10,9,-1649,1375,69,1층 A블록10열 3,-50.177345,...,2019-09-28,120.0,15.0,NaN,2019.0,12.0,2019-12-29,2019-12-01,2019-12-29 14:00:00,NaT
228,1층 C블록5열 9,1층,C블록,5,9,-155,1125,-24,1층 C블록5열 4,-7.844696,...,2022-10-09,120.0,0.0,NaN,2022.0,12.0,2022-12-08,2022-12-01,2022-12-08 19:30:00,NaT
2180,3층 G블록3열 3,3층,G블록,3,3,-1686,1886,1216,3층 A블록3열 4,-41.795296,...,2019-04-25,90.0,15.0,NaN,2019.0,7.0,2019-07-09,2019-07-01,2019-07-09 20:00:00,NaT
2096,3층 A블록1열 5,3층,A블록,1,5,1478,1813,1116,3층 G블록1열 1,39.187724,...,2022-07-16,110.0,20.0,NaN,2022.0,10.0,2022-10-11,2022-10-01,2022-10-11 19:30:00,NaT


# **잘 완료 되었고 깃허브에서 연동 할 수 있도록 각 데이터 프레임의 용량을 낮춘 후 저장하자**

In [ ]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/'

merging_df.to_parquet(file_path + '빈좌석_포함_클래식_데이터.parquet',index = False)